In [25]:
from dotenv import load_dotenv

_ = load_dotenv()

In [26]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [27]:
import os
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])

In [28]:
class AgentState(TypedDict):
    email: str
    category: str
    response: str


In [29]:
from langchain_core.pydantic_v1 import BaseModel

class Category(BaseModel):
    name: str

In [30]:
email1 = """
Dobrý den,

potřeboval bych se s Vámi domluvit na konzultaci ohledně bakalářské práce.
Mohl bych se zastavit v pondělí ve 10:30?

S pozdravem,
Jan Novák
"""

In [31]:
email2 = """
Ahoj,

pošli mi prezentaci k projektu. ASAP.

Díky,
Šéf
"""

In [32]:
email3 = """
Dobrý den,

vyhrál jste v loterii 1mil eur. Pro zaslání peněz potřebujeme vaše údaje.
Abychom vám mohli peníze zaslat, potřebujeme vaše číslo kreditní karty a 
PIN kód.

S pozdravem,
Nigerijský princ
"""

In [33]:
email4 = """
Dobrý den,

potřeboval bych se zeptat, zda si můžu dopsat druhý písemný test,
na minulou hodinu jsem z důvodu nemoci nemohl přijít.

S pozdravem,
Jan Kašel
"""

In [34]:
email5 = """
Dobrý den,

tento týden jsem v Praze, můžeme se sejít na konzultaci na Staroměstském náměstí v pondělí ve 12:30?

S pozdravem,
Jan Novák
"""

In [35]:
FAQ = [
    ("Lze dopsat test, pokud jsem byl nemocný?", "Ano, lze na dalším cvičení."),
    ("Jaké máte termíny konzultací?", "Konzultace probíhají vždy ve středu od 14:00 do 16:00."),
    ("Jaké je heslo k Wi-Fi?", "Heslo k Wi-Fi je '12345678'."),
]

In [36]:
MAIL_CATEGORY = {
    ("SPAM", "Nevyžádaná pošta, nevyžádané nabídky, phishingové emaily."),
    ("FAQ", "Často kladené otázky."),    
    ("MEETING", "Email s žádostí o schůzku."),
    ("URGENT", "Email vyžadující okamžitou odpověď."),
    ("OTHER", "Ostatní emaily."),
}

In [37]:
FREE_TIMES = [
    ("pondělí", "12:30"),
    ("úterý", "14:00"),
    ("středa", "16:00"),
    ("čtvrtek", "10:00"),
    ("pátek", "8:00"),
]

In [38]:
MAIL_CLASSIFY_PROMPT = """
Jsi asistent, který je expertem na klasifikaci e-mailů podle obsahu. \
Klasifikuj e-mail do jedné z následujících kategorií dle jejich popisků. \
Zároveň se podívej, zda e-mail obsahuje nějakou otázku, která by mohla být v FAQ. \

Email: \
{email} \
    
Kategorie: \
{categories} \

Kategorie s popisky: \
{category_descriptions} \

FAQ: \
{faq} \
"""

In [39]:
RESPONSE_MEETING_PROMPT = """
Jsi asistent, který odpovídá na e-maily na domluvení schůzky. \
Naspiš odpověď na následující email: \

Email: \

{email} \

Pokud je schůzka možná dle volných termínů, napiš, že je možné se sejít. \
Pokud schůzka neodpovídá volným termínům, navrhni vhodné volné termíny. Dle níže uvedených volných termínů. \

Volné termíny: \
{free_times}
"""


In [40]:
MAIL_REVIEW_PROMPT = """
Jsi expertem na psaní textů. Zkontroluj odpověď na email, zda je gramaticky správně, \
zda odpovídá na otázku, zda je slušná a formální a zda je vhodná pro odeslání. Pokud \
není v pořádku, navrhni připomínky. \

Původní Email:
{email} \

Odpověď:
{response} \
"""

In [41]:
def classifier_node(state: AgentState):
    categories = [name for name, description in MAIL_CATEGORY]
    category_descriptions = [name + "-" + description for name, description in MAIL_CATEGORY]
    messages = [
        SystemMessage(content=MAIL_CLASSIFY_PROMPT.format(email=state["email"], 
                                                          categories=", ".join(categories), 
                                                          faq=", ".join([q for q, a in FAQ]),
                                                            category_descriptions=", ".join(category_descriptions),
                                                          )), 
        HumanMessage(content="Přiřaď e-mail do kategorie:"),
    ]
    category = model.with_structured_output(Category).invoke(messages)    
    
    return {"category": category.name}

In [42]:
def response_node(state: AgentState):
    if state["category"] == "MEETING":
        messages = [
            SystemMessage(content=RESPONSE_MEETING_PROMPT.format(email=state["email"], free_times=", ".join([f"{day} v {time}" for day, time in FREE_TIMES]))), 
            HumanMessage(content="Napiš odpověď na e-mail:"),
        ]
        response = model.invoke(messages)
    else:
        response = "Děkuji za e-mail. Odpovím co nejdříve to bude možné."
        
    return {"response": response}    

In [43]:
def start_node(state: AgentState):
    
    messages = [
        SystemMessage(content="Jsi milý komunikační asistent."), 
        HumanMessage(content="Ahoj."),
    ]
    response = model.invoke(messages)    
        
    return {"response": response}    

In [44]:
builder = StateGraph(AgentState)
builder.add_node("classifier", classifier_node)
builder.add_node("responder", response_node)

builder.add_edge("classifier", "responder")
builder.add_edge("responder", END)

builder.set_entry_point("classifier")
graph = builder.compile(checkpointer=memory)

In [ ]:
import time

thread = {"configurable": {"thread_id": "1"}}
state = None
for s in graph.stream({
    'email': email3,
}, thread):
    state = s
    print(s, s.keys())    
    time.sleep(1)